In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
df.shape

In [ ]:
#make trg

def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)

In [ ]:
#drop like orig sol
df.drop(['order_id' , 'line_item_type_id', 'integration_type_id' , 'revenue_share_percent'], axis = 1, inplace=True)

In [ ]:
perc95 = df['CPM'].describe([.95])[5]
print(perc95)

split_date = '2019-06-22'

In [ ]:
df = df[(df['CPM'] >= 0) & (df['CPM'] <= perc95)]

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
for col in df.columns:
    print(col, len(df[col].unique()))

In [ ]:
cat_feats = []
for col in df.columns:
    if col.find('id') >= 0:
        print(col, len(df[col].unique()))
        cat_feats.append(col)

In [ ]:
set(df.columns) - set(cat_feats)

In [ ]:
fin_cols = list(set(df.columns) - set(['CPM', 'date', 'total_revenue']))

In [ ]:
train = df[df['date'] < split_date]
test = df[df['date'] >= split_date]

In [ ]:
from catboost import CatBoostRegressor

clf = CatBoostRegressor(iterations = 300, 
                       has_time = True, cat_features = cat_feats,
                       task_type = 'CPU', eval_metric = "RMSE", 
                       random_state = 1337)

clf.fit(train[fin_cols], train['CPM'], silent = True)

In [ ]:
from sklearn.metrics import mean_squared_error

pred = clf.predict(test[fin_cols])
print('Is score better?' , mean_squared_error(test['CPM'], pred) < 4850)
mean_squared_error(test['CPM'], pred)

In [ ]:
impotances = pd.DataFrame([clf.feature_names_, clf.feature_importances_]).T
impotances.sort_values(by=1, ascending=False).head(20)

### Get some features from top

In [ ]:
el = 5
fin_cols2 = impotances[0].head(el).values
cat_feats2 = set(impotances[0].head(el).values) & set(cat_feats)

In [ ]:
clf2 = CatBoostRegressor(iterations = 300, 
                       has_time = True, cat_features = cat_feats2,
                       task_type = 'CPU', eval_metric = "RMSE", 
                       random_state = 1337)

clf2.fit(train[fin_cols2], train['CPM'], silent = True)

In [ ]:
pred2 = clf2.predict(test[fin_cols2])
print('Is score better?' , mean_squared_error(test['CPM'], pred2) < 4850)
mean_squared_error(test['CPM'], pred2)

In [ ]:
print('Curr best MSE =' , mean_squared_error(test['CPM'], pred))